# Logistics Regression Explaination version (Multiclass: softmax)

### Penulis: Patuan P. Tampubolon

Package yang perlu kita gunakan.

In [1]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

# 1. Pendahuluan

Logistic regression merupakan algoritma klasifikasi. Ini berarti dependent variabelnya (targetnya) berupa nilai diskrit. Ada tiga jenis klasifikasi yang dapat diselesaikan, yaitu: 
- binary: contoh targetnya (0 dan 1), (kucing dan anjing), (ya dan tidak), dan lain-lain
- multy: contoh targetnya (0,1 dan 2), (cerah, hujan, dan mendung), dan lain-lain
- ordinal: contoh targetnya (mudah, sedang, dan sulit), (tidak setuju, setuju dan sangat setuju), (tidak normal, normal, luar biasa), dan lain-lain.

Pada pembelajaran ini akan dipaparkan implementasi untuk permasalahan multiclassification dengan pendekatan one-vs-rest.

Dataset yang digunakan sebagai contoh adalah Iris dataset.

# 2. Dataset

In [2]:
iris = datasets.load_iris()

Dataset iris merupakan dataset yang mengelompokkan spesies bunga iris menjadi tiga kelas. Yaitu:

In [3]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

Untuk mengelompokkan bunga tersebut, ada 4 ciri2 yang digunakan, yaitu:

In [4]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

Pada machine learning, fitur-fiturnya akan dijadikan predictor (X) , dan kelasnya sebagai target (y). Implementasinya pada python adalah sebagai berikut ini.

In [5]:
X = iris.data
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [6]:
X.shape

(150, 4)

In [7]:
y = iris.target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Sekarang tentukan X dan y untuk training dan testing.

# 3. Tools yang diperlukan

## 3.1. Fungsi aktifasi Softmax

Fungsi softmax merupakan fungsi general dari fungsi sigmoid. Fungsi softmax dirumuskan sebagai berikut (Eq. 1).

## $f(x) = \frac{e^{(\theta^{(k)T}x^{i})}}{\sum_{j=1}^{K}e^{(\theta{(j)T}x^{(i)})}}$ ----> (1)

Jika diimplemantasikan kedalam bahasa python maka fungsi tersebut akan menjadi seperti berikut ini.

In [8]:
def softmax(z):
    z -= np.max(z)
    sm = (np.exp(z).T / np.sum(np.exp(z),axis=1)).T
    return sm

In [9]:
nilai = np.array([[-2,-1,0,1,2],[-1,-1,1,1,2],[0,1,0,1,2]])
peluang = softmax(nilai)

In [10]:
peluang

array([[0.01165623, 0.03168492, 0.08612854, 0.23412166, 0.63640865],
       [0.02712699, 0.02712699, 0.20044288, 0.20044288, 0.54486025],
       [0.06745081, 0.1833503 , 0.06745081, 0.1833503 , 0.49839779]])

In [11]:
peluang[0,:].sum()

1.0

## 3.2. Relabel Target

Karena kita akan membuat logistic regression untuk multiclass kita perlu melabel kembali label yang sudah ada menjadi sebanyak kelasnya.

In [12]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [13]:
label = list(set(y))
label

[0, 1, 2]

In [14]:
relabeled_data = np.zeros(len(y)*len(label)).reshape(len(y),len(label))
relabeled_data[0:20,:]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [15]:
for i in range(len(label)):
    relabeled_data[y==label[i],i] = 1

In [16]:
relabeled_data[0:20,:]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [17]:
def relabel_data(y):
    label = list(set(y))
    relabeled_data = np.zeros(len(y)*len(label)).reshape(len(y),len(label))
    for i in range(len(label)):
        relabeled_data[y==label[i],i] = 1
    return relabeled_data

In [18]:
rbl = relabel_data(y)
rbl[0:50]
#rbl[50:100]
#rbl[100:150]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

## 3.3. Cost Function

Untuk menentukan seberapa baik kita melakukan prediksi, kita perlu membuat sebuah cost function yang disebut dengan Cross-entropy loss. Seperti yang dirumuskan di bawah ini:

#### $J(\theta) = - \sum_{i=1}^{m}[y^{(i)}\log (h_{\theta}(x^{(i)})) + (1-y^{(i)})\log(1-h_{\theta}(x^{(i)})]$ ----> (4)

Eq. 4 dapat dihitung secara vektorisasi dengan eq. 5 dibawah ini.

#### $h = g(X\theta)$
#### $J(\theta) = (-y^{T}\log (h) - (1-y)^{T}\log (1-h))$ -----> (5)

Eq. 5 dapat diimplementasikan sebagai berikut.

In [19]:
y1 = rbl
y1.shape

(150, 3)

In [20]:
theta0 = np.random.rand(X.shape[1], len(label)) # theta diinisialisasi dengan angka 0
theta0

array([[0.91503961, 0.3009641 , 0.27990034],
       [0.52748896, 0.52055931, 0.39603925],
       [0.73070943, 0.72976057, 0.33586395],
       [0.10944347, 0.34986649, 0.80489196]])

In [21]:
z0 = np.dot(X, theta0)
z0[0:10]

array([[7.55779524, 4.44851256, 3.44481701],
       [7.11104284, 4.12804009, 3.19081731],
       [6.96046177, 4.09898307, 3.1804587 ],
       [6.9623508 , 4.16278284, 3.18003753],
       [7.51904018, 4.47047208, 3.4564309 ],
       [8.28440423, 5.03592698, 3.94894038],
       [7.05847089, 4.28096123, 3.34575211],
       [7.48661333, 4.43933627, 3.41080944],
       [6.60077414, 3.92550211, 3.01126322],
       [7.22591833, 4.21808542, 3.18351844]])

In [22]:
h0 = softmax(z0)
h0[0:20]

array([[0.94251423, 0.0420672 , 0.01541857],
       [0.93416285, 0.04730652, 0.01853063],
       [0.92592013, 0.05294795, 0.02113192],
       [0.9228444 , 0.05614248, 0.02101313],
       [0.93929271, 0.04454763, 0.01615966],
       [0.95063442, 0.03691627, 0.01244932],
       [0.92029825, 0.05723629, 0.02246546],
       [0.93943763, 0.04461207, 0.0159503 ],
       [0.91199277, 0.06282544, 0.02518179],
       [0.93724761, 0.04629873, 0.01645365],
       [0.95262707, 0.03531251, 0.01206042],
       [0.93264407, 0.05007224, 0.01728369],
       [0.93258712, 0.04902474, 0.01838814],
       [0.9074613 , 0.06486679, 0.02767191],
       [0.96193549, 0.02784175, 0.01022275],
       [0.95862375, 0.03086194, 0.01051431],
       [0.9486006 , 0.03685127, 0.01454813],
       [0.9405071 , 0.04299908, 0.01649382],
       [0.95989148, 0.03028859, 0.00981994],
       [0.94177379, 0.0429635 , 0.0152627 ]])

In [23]:
np.log(h0)[0:10,:]

array([[-0.05920426, -3.16848695, -4.1721825 ],
       [-0.0681045 , -3.05110725, -3.98833002],
       [-0.0769673 , -2.938446  , -3.85697037],
       [-0.08029464, -2.8798626 , -3.86260791],
       [-0.06262813, -3.11119622, -4.1252374 ],
       [-0.05062571, -3.29910296, -4.38608956],
       [-0.08305748, -2.86056714, -3.79577626],
       [-0.06247384, -3.1097509 , -4.13827773],
       [-0.09212321, -2.76739525, -3.68163413],
       [-0.06480777, -3.07264068, -4.10720767]])

In [24]:
np.sum(-y1 * np.log(h0) - (1 - y1) * np.log(1 - h0),axis=1)

array([ 0.11772059,  0.13527103,  0.15272692,  0.15931174,  0.12449014,
        0.10076809,  0.16471873,  0.12419053,  0.1825132 ,  0.12880308,
        0.09661659,  0.13853577,  0.13861925,  0.19223264,  0.07731993,
        0.08417479,  0.10496986,  0.12191837,  0.08156028,  0.1192843 ,
        0.09583132,  0.1239849 ,  0.16471315,  0.12775658,  0.13474426,
        0.12483831,  0.13178599,  0.10976972,  0.11130943,  0.14810255,
        0.14009541,  0.10483437,  0.10379262,  0.09016822,  0.13334336,
        0.12918437,  0.0935481 ,  0.12766219,  0.18373783,  0.11698637,
        0.13080125,  0.18492677,  0.18226978,  0.14084172,  0.11880276,
        0.14871782,  0.11416033,  0.16033339,  0.10260406,  0.12594311,
        7.86450447,  7.08165098,  7.69922063,  6.06815317,  7.19393912,
        6.34825359,  6.9248999 ,  5.49297061,  7.42292511,  5.67709911,
        5.60676305,  6.46545696,  6.81006211,  6.78098196,  6.17691772,
        7.48368782,  6.13371045,  6.59638085,  6.81172209,  6.29

In [25]:
np.mean(np.sum(-y1 * np.log(h0) - (1 - y1) * np.log(1 - h0),axis=1))

5.203115389858735

In [26]:
def cost(h, y):
    return np.mean(np.sum(-y * np.log(h) - (1 - y) * np.log(1 - h),axis=1))

In [27]:
res = cost(h0,y1)
res

5.203115389858735

## 3.4. Gradient Descent (minimizing cost)

Gradient descent digunakan untuk mengatur nilai theta. Adapun algoritma gradient descent dapat dilakukan dalam 3 tahap, yaitu:
1. hitung rata-rata gradien
2. kalikan dengan nilai learning rate (lr atau alpha)
3. Kurangi dari bobot sebelumnya

Implementasi dari gradient descent dapat dilihat dibawah ini.

Membangun theta atau bobot

In [28]:
theta = np.zeros(X.shape[1]*len(label)).reshape(X.shape[1],len(label)) # theta diinisialisasi dengan angka 0
theta

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

Perkalian antara X dan bobotnya

In [29]:
z = np.dot(X, theta)
z[0:10,:]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

Menghitung nilai sigmoid dari z

In [30]:
h = softmax(z)
h[0:10,:]

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

Menghitung rata-rata gradient

In [31]:
gradient = np.dot(X.T, (h - y1)) / y1.size # rata-rata gradient
gradient

array([[ 0.09303704, -0.0102963 , -0.08274074],
       [-0.04118519,  0.03192593,  0.00925926],
       [ 0.25511111, -0.05577778, -0.19933333],
       [ 0.10592593, -0.01407407, -0.09185185]])

Menghitung bobot yang baru jika nilai learning rate (alpha) adalah 0.1

In [32]:
alpha = 0.1
theta = theta - alpha * gradient
theta

array([[-0.0093037 ,  0.00102963,  0.00827407],
       [ 0.00411852, -0.00319259, -0.00092593],
       [-0.02551111,  0.00557778,  0.01993333],
       [-0.01059259,  0.00140741,  0.00918519]])

In [33]:
def grad_desc(X, y, theta, alpha, num_iter):
    theta_new = np.copy(theta)

    for i in range(num_iter):
        z = np.dot(X, theta)
        h = softmax(z)
        gradient = np.dot(X.T, (h - y)) / y.size
        theta_new = theta_new - alpha * gradient
    print('iter num: ', i, 'cost : ', cost(h, y).mean(), 'theta: ', theta_new)
    return theta_new

In [34]:
gr = grad_desc(X,y1,theta,0.1,10)

iter num:  9 cost :  1.8559553361616046 theta:  [[-0.00409991  0.00337431  0.0007256 ]
 [ 0.09318809 -0.03877484 -0.05441325]
 [-0.20944037  0.05514697  0.1542934 ]
 [-0.09271825  0.01336312  0.07935513]]


Ketika kita membuat memasukan features kita perlu memasukkan 

## 3.5 Fungsi prediksi

In [35]:
def predict(Xt, theta):
    z = np.dot(Xt, theta)
    return softmax(z)

In [36]:
theta

array([[-0.0093037 ,  0.00102963,  0.00827407],
       [ 0.00411852, -0.00319259, -0.00092593],
       [-0.02551111,  0.00557778,  0.01993333],
       [-0.01059259,  0.00140741,  0.00918519]])

In [37]:
pred = predict(X,theta)
pred[0:10,:]

array([[0.31002495, 0.33351517, 0.35645988],
       [0.30997027, 0.33398658, 0.35604315],
       [0.31166068, 0.33358634, 0.35475298],
       [0.31018118, 0.33397255, 0.35584627],
       [0.31045756, 0.33339178, 0.35615067],
       [0.30649975, 0.33367075, 0.35982949],
       [0.31105716, 0.33354463, 0.35539822],
       [0.30937142, 0.33374752, 0.35688106],
       [0.31134503, 0.33398317, 0.35467179],
       [0.30962083, 0.33400322, 0.35637595]])

In [38]:
pred[0,:].sum()

1.0

# 4. Langkah pembuatan model langkah per langkah

## 4.1 Split Train dan Test

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

## 4.2. Menyesuaikan data pada model

Model logistic regression yang kita akan gunakan merupakan model linear seperti yang ditunjukkan pada gambar dibawah ini. Dengan Demikian kita harus menambahkan angka 1 pada X, menginisialisasi $\theta$, dan merelabel y.

![image](linearModel.png)

Catatan: di dalam coding menggunakan $\theta$, gambar diatas menggunakan $\beta$. Mereka dimaksudkan sama, yaitu bobot.

In [40]:
X[0:20,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3]])

In [41]:
satu = np.ones((len(X_train))).reshape(-1,1)
satu[0:20,:]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [42]:
X_new = np.concatenate((satu, X_train),axis = 1)
X_new[0:10,:]

array([[1. , 6.3, 2.9, 5.6, 1.8],
       [1. , 6.7, 3.1, 5.6, 2.4],
       [1. , 5.4, 3. , 4.5, 1.5],
       [1. , 7. , 3.2, 4.7, 1.4],
       [1. , 6. , 3. , 4.8, 1.8],
       [1. , 4.6, 3.1, 1.5, 0.2],
       [1. , 6.7, 3.3, 5.7, 2.5],
       [1. , 5.5, 2.5, 4. , 1.3],
       [1. , 4.9, 2.4, 3.3, 1. ],
       [1. , 5.1, 3.8, 1.6, 0.2]])

In [43]:
def addintercept(x):
    return np.concatenate((np.ones((len(x))).reshape(-1,1), x),axis = 1)

In [44]:
X_new = addintercept(X_train)
X_new[0:10,:]

array([[1. , 6.3, 2.9, 5.6, 1.8],
       [1. , 6.7, 3.1, 5.6, 2.4],
       [1. , 5.4, 3. , 4.5, 1.5],
       [1. , 7. , 3.2, 4.7, 1.4],
       [1. , 6. , 3. , 4.8, 1.8],
       [1. , 4.6, 3.1, 1.5, 0.2],
       [1. , 6.7, 3.3, 5.7, 2.5],
       [1. , 5.5, 2.5, 4. , 1.3],
       [1. , 4.9, 2.4, 3.3, 1. ],
       [1. , 5.1, 3.8, 1.6, 0.2]])

In [45]:
y_new = relabel_data(y_train)
y_new[0:10,:]

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [61]:
X_new.shape[1]

5

## 4.3 Menginisalisasi Theta

In [46]:
theta = np.zeros(X_new.shape[1]*len(label)).reshape(X_new.shape[1],y_new.shape[1]) # theta diinisialisasi dengan angka 0
theta

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

## 4.4. menghitung z

In [47]:
z = np.dot(X_new, theta)
z[0:10,]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

## 4.5. menghitung h

In [48]:
h = softmax(z)
h[0:10,:]

array([[0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333]])

## 4.6.  Menghitung gradient

In [49]:
#gradient = np.dot(X.T, (h - y)) / y.size
gradient = np.dot(X_new.T, (h - y_new)) / y_new.shape[0] # rata-rata gradient
gradient

array([[-2.50000000e-02,  2.50000000e-02, -4.44089210e-17],
       [ 1.39444444e-01,  1.10277778e-01, -2.49722222e-01],
       [-2.02500000e-01,  1.68333333e-01,  3.41666667e-02],
       [ 6.99444444e-01, -8.47222222e-02, -6.14722222e-01],
       [ 3.01944444e-01, -1.63888889e-02, -2.85555556e-01]])

## 4.7. Update theta

In [50]:
alpha = 0.1

In [51]:
theta = theta - alpha * gradient
theta

array([[ 2.50000000e-03, -2.50000000e-03,  4.44089210e-18],
       [-1.39444444e-02, -1.10277778e-02,  2.49722222e-02],
       [ 2.02500000e-02, -1.68333333e-02, -3.41666667e-03],
       [-6.99444444e-02,  8.47222222e-03,  6.14722222e-02],
       [-3.01944444e-02,  1.63888889e-03,  2.85555556e-02]])

## 4.8. Menghitung cost

In [52]:
cost = cost(h, y_new).mean()
cost

1.9095425048844383

Pada iterasi pertama cost pada semua kelas sama, karena h0, h1, dan h2 masih sama yaiut semua entrinya 0. Nilainya akan berubah pada iterasi ke-2 dan seterusnya.

## 4.9. Melakukan prediksi

In [53]:
pred = predict(X_new,theta)
pred

array([[0.19023795, 0.28444569, 0.52531636],
       [0.18497477, 0.28015306, 0.53487217],
       [0.21683495, 0.29273315, 0.49043191],
       [0.20797075, 0.2835171 , 0.50851215],
       [0.20622798, 0.28828   , 0.50549202],
       [0.29552644, 0.30072863, 0.40374493],
       [0.18337805, 0.27873872, 0.53788324],
       [0.22518593, 0.29598529, 0.47882878],
       [0.24385335, 0.3011074 , 0.45503925],
       [0.2950669 , 0.29530059, 0.40963251],
       [0.30038482, 0.2970269 , 0.40258827],
       [0.18343285, 0.28129587, 0.53527128],
       [0.29602247, 0.28981768, 0.41415986],
       [0.22387232, 0.29346524, 0.48266244],
       [0.19826043, 0.28795805, 0.51378153],
       [0.29503811, 0.29953947, 0.40542242],
       [0.18638587, 0.2833886 , 0.53022553],
       [0.19600619, 0.28788312, 0.51611068],
       [0.2127528 , 0.28915299, 0.49809422],
       [0.30194601, 0.29128385, 0.40677013],
       [0.29070308, 0.29960624, 0.40969067],
       [0.29570823, 0.29993474, 0.40435704],
       [0.

## 4.10. Mencari nilai prediksi yang paling maksimal dan mengkonversi menjadi kelas

In [54]:
pred_max = np.zeros(len(pred))
pred_max

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [55]:
pred [1,:] == max(pred[1,:])

array([False, False,  True])

In [56]:
rr = np.where(pred [1,:] == max(pred[1,:]))
rr[0][0]

2

In [57]:
for i in range(len(pred)):
    rr = np.where(pred [i,:] == max(pred[i,:]))
    pred_max[i] = rr[0][0]

In [58]:
pred_max

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2.])

In [59]:
y_train

array([2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 0, 2, 0, 1, 2, 0, 2, 2, 1, 0, 0, 0,
       2, 2, 1, 0, 0, 2, 1, 0, 1, 2, 0, 1, 2, 2, 2, 2, 0, 1, 0, 2, 2, 0,
       1, 2, 0, 0, 2, 2, 1, 2, 1, 0, 1, 2, 2, 1, 1, 2, 2, 0, 2, 0, 1, 2,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 2, 0, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 2, 0, 2,
       2, 1, 1, 0, 2, 1, 0, 0, 2, 0])